In [ ]:
import sys
import os
this_dir = ".."
if this_dir not in sys.path:
    sys.path.append(this_dir)

from rnn_utils import DiagnosesDataset, split_dataset, MYCOLLATE
from rnn_utils import train_one_epoch, eval_model

from mourga_variational.variational_rnn import VariationalRNN


import torch
from torch.utils.data import Dataset, DataLoader, random_split

from sklearn.model_selection import ParameterGrid, ParameterSampler

import numpy as np

import wandb

# Reproducibility

In [7]:
# Reproducibility
np.random.seed(546)
torch.manual_seed(546)
torch.cuda.manual_seed(546)

# Create dataset

In [8]:
grouping = 'ccs'
batch_size=64

In [51]:
dataset = DiagnosesDataset('data/model_data.json',grouping)
test_size = 0.15
eval_size=0.15
eval_size_corrected = eval_size/(1-test_size)

whole_train_dataset,test_dataset = split_dataset(dataset,test_size)
train_dataset, val_dataset = split_dataset(whole_train_dataset,eval_size_corrected)

len(whole_train_dataset)
len(train_dataset)
len(val_dataset)
len(test_dataset)

whole_train_dataset = DataLoader(whole_train_dataset,batch_size=batch_size,collate_fn=MYCOLLATE(dataset),shuffle=True)
train_dataloader = DataLoader(train_dataset,batch_size=batch_size,collate_fn=MYCOLLATE(dataset))#,shuffle=True)
val_dataloader = DataLoader(val_dataset,batch_size=batch_size,collate_fn=MYCOLLATE(dataset)) #batch_size here is arbitrary and doesn't affect total validation speed
test_dataloader = DataLoader(test_dataset,batch_size=batch_size,collate_fn=MYCOLLATE(dataset))

6375

5250

1125

1124

In [52]:
train_dataloader = DataLoader(train_dataset,batch_size=batch_size,collate_fn=MYCOLLATE(dataset))#,shuffle=True)
val_dataloader = DataLoader(val_dataset,batch_size=batch_size,collate_fn=MYCOLLATE(dataset)) #batch_size here is arbitrary and doesn't affect total validation speed
test_dataloader = DataLoader(test_dataset,batch_size=batch_size,collate_fn=MYCOLLATE(dataset))

In [53]:
next(iter(train_dataloader))['train_sequences']['original'][0][0]
next(iter(val_dataloader))['train_sequences']['original'][0][0]

[257,
 98,
 131,
 259,
 101,
 59,
 106,
 109,
 238,
 47,
 110,
 49,
 83,
 52,
 53,
 115,
 155,
 95]

[2, 130, 3, 98, 2616, 237, 657, 149, 152, 122]

# Define model

## Hyperparameters

In [16]:
input_size = next(iter(train_dataloader))['target_sequences']['sequence'].shape[2]
hidden_size = 100
num_layers = 1
n_labels = input_size
model_type = 'gru'

## Init model

In [6]:
model = RNN(input_size=input_size,
              hidden_size=hidden_size,
              num_layers=num_layers,
              n_labels=n_labels,
              model=model_type)

# Define Training

In [7]:
opt = torch.optim.Adam(model.parameters(), lr=0.02)
criterion = torch.nn.BCEWithLogitsLoss()
epochs = 10

# Train

In [16]:
for epoch in range(1,2+1):
    loss = train_one_epoch(model,train_dataloader, epoch, criterion, opt)
    print({'epoch':epoch,'train_loss':loss})
    print(eval_model(model,val_dataloader,dataset,criterion,epoch,'val',only_loss=False))

{'epoch': 1, 'train_loss': 0.06310431027709275}
{'name': 'val', 'epoch': 1, 'loss': 0.055693817211521994, '1 adm': {'recall10': {'mean': 0.4383168086764939, 'std': 0.20194492969027655, 'n': 1125}, 'recall20': {'mean': 0.6003363384028486, 'std': 0.19180999013921282, 'n': 1125}, 'recall30': {'mean': 0.7022388516986957, 'std': 0.1738569564575414, 'n': 1125}}, '2 adm': {'recall10': {'mean': 0.4324873976918547, 'std': 0.16014301456855048, 'n': 342}, 'recall20': {'mean': 0.6070942967310907, 'std': 0.1580827383618025, 'n': 342}, 'recall30': {'mean': 0.7079473060972277, 'std': 0.15124369277599858, 'n': 342}}, '3 adm': {'recall10': {'mean': 0.4250083665874939, 'std': 0.1499507856955461, 'n': 143}, 'recall20': {'mean': 0.6089853391388756, 'std': 0.14788611278223662, 'n': 143}, 'recall30': {'mean': 0.7145667129901153, 'std': 0.1362589636676128, 'n': 143}}, '>3 adm': {'recall10': {'mean': 0.48252846941024563, 'std': 0.1994391330440701, 'n': 248}, 'recall20': {'mean': 0.6615573569722966, 'std': 0.1

# Now for wandb

In [63]:
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

## define data and train conditions

In [21]:
input_size = next(iter(train_dataloader))['target_sequences']['sequence'].shape[2]
n_labels = input_size

criterion = torch.nn.BCEWithLogitsLoss()

## Define search space

In [69]:
hyperparameters = {
    'hidden_size':[50,100,input_size], #-1 is to have the same as input size
    'num_layers':[1,2],
    'lr':[0.01],
    'model':['RNN','GRU','LSTM'],
    'dropout':[0.1,0.2,0.3],
    'batch_size':[64],
    'name':['normal_config']
}

meta_parameters = {
    'epochs':15
}

params = ParameterGrid(hyperparameters)
print(f'params:',len(params))

random_params = ParameterSampler(params.param_grid,n_iter=len(params)-1,random_state=231)
next(iter(random_params))

params: 54


{'num_layers': 2,
 'name': 'normal_config',
 'model': 'GRU',
 'lr': 0.01,
 'hidden_size': 50,
 'dropout': 0.2,
 'batch_size': 64}

## Run models

In [70]:
for param_set in params:
    config = {**param_set, 
              **meta_parameters}
    
    wandb.init(
        project="idrop_vs_rnn", 
        config=config
    )
    
    whole_train_dataset = DataLoader(whole_train_dataset,batch_size=config['batch_size'],collate_fn=MYCOLLATE(dataset),shuffle=True)
    train_dataloader = DataLoader(train_dataset,batch_size=config['batch_size'],collate_fn=MYCOLLATE(dataset),shuffle=True)
    val_dataloader = DataLoader(val_dataset,batch_size=config['batch_size'],collate_fn=MYCOLLATE(dataset)) #batch_size here is arbitrary and doesn't affect total validation speed
    
    model = VariationalRNN(input_size=input_size,
                          hidden_size=config['hidden_size'],
                          num_layers=config['num_layers'],
                          n_labels=n_labels,
                          rnn_type=config['model'],
                          dropouti=config['dropout'],
                          dropoutw=config['dropout'],
                          dropouto=config['dropout'])
    
    optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])
    
    for epoch in range(1,config['epochs']+1):
        loss = train_one_epoch(model,train_dataloader,epoch,criterion,optimizer)
        wandb.log({"loss":loss})
    
    # eval model on validation data
    train_results = eval_model(model,train_dataloader,dataset, criterion, epoch, 'train_last')
    val_results = eval_model(model,val_dataloader,dataset, criterion, epoch, 'validation')
    wandb.log({'train_loss':train_results['loss'],
               'train_recall@30':train_results['last adm']['recall30']['mean'],
               'val_loss':val_results['loss'],
               'recall@10':val_results['last adm']['recall10']['mean'],
               'recall@20':val_results['last adm']['recall20']['mean'],
               'recall@30':val_results['last adm']['recall30']['mean']
              })

wandb: wandb version 0.12.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


loss,0.0363
_runtime,30
_timestamp,1631882898
_step,15
train_loss,0.03512
train_recall@30,0.7196
val_loss,0.03928
recall@10,0.41005
recall@20,0.57889
recall@30,0.68623


loss,█▅▃▃▂▂▂▁▁▁▁▁▁▁▁
_runtime,▁▂▂▂▃▃▄▄▄▅▅▅▆▆▆█
_timestamp,▁▂▂▂▃▃▄▄▄▅▅▅▆▆▆█
_step,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
train_loss,▁
train_recall@30,▁
val_loss,▁
recall@10,▁
recall@20,▁
recall@30,▁


wandb: wandb version 0.12.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


loss,0.03944
_runtime,29
_timestamp,1631882931
_step,15
train_loss,0.03848
train_recall@30,0.6369
val_loss,0.04184
recall@10,0.34264
recall@20,0.50852
recall@30,0.6245


...

loss,█▅▃▃▂▂▂▁▁▁▁▁▁▁▁
_runtime,▁▁▂▂▂▃▃▃▄▄▅▅▆▇▇█
_timestamp,▁▁▂▂▂▃▃▃▄▄▅▅▆▇▇█
_step,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
train_loss,▁
train_recall@30,▁
val_loss,▁
recall@10,▁
recall@20,▁
recall@30,▁


wandb: wandb version 0.12.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


loss,0.03823
_runtime,48
_timestamp,1631885286
_step,15
train_loss,0.03506
train_recall@30,0.75151
val_loss,0.03942
recall@10,0.43084
recall@20,0.60073
recall@30,0.70328


loss,█▅▃▃▂▂▂▁▁▁▁▁▁▁▁
_runtime,▁▂▂▃▃▃▄▄▅▅▅▆▆▆▇█
_timestamp,▁▂▂▃▃▃▄▄▅▅▅▆▆▆▇█
_step,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
train_loss,▁
train_recall@30,▁
val_loss,▁
recall@10,▁
recall@20,▁
recall@30,▁


wandb: wandb version 0.12.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


# best model

- lr: 0.03
- model_type = 'lstm'
- num_layers = 1
- hidden_size = 50
- epochs: 15